In [ ]:
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
%matplotlib inline
from analysis.lib.purification import purify_analysis as pa; reload(pa)

In [ ]:
import matplotlib as mpl


mpl.rcParams['font.family'] = 'sans-serif'
mpl.rcParams['font.sans-serif'] = 'Helvetica'

In [ ]:
# time stamp defs

#### compose the timestamps, used order is: X,mX,Y,mY,Z,mZ
### shown data is for pi/6 in the LDE element
daystamp = '20160817'
LT3_tstamp_list = ['140755','151440','142421','143655','144136','144736']
LT3_tstamp_list = [daystamp+'_'+s for s in LT3_tstamp_list]
LT3_ssro = r'K:\ns\qt\Diamond\Projects\Purification\Data\SSROs_lt3\combined_ssro_MWInit'
LT4_tstamp_list = ['172831','173338','175151','175644','150853','151312']
LT4_tstamp_list = [daystamp+'_'+s for s in LT4_tstamp_list]
LT4_ssro = r'K:\ns\qt\Diamond\Projects\Purification\Data\SSROs_lt4\combined_ssro'
base_folder_lt3 = r'K:\ns\qt\Diamond\Projects\Purification\Data\pur_lt3_local'
base_folder_lt4 = r'K:\ns\qt\Diamond\Projects\Purification\Data\pur_lt4_local'

In [ ]:
#get data def
reload(mbi)
def do_carbon_ROC(f,f_u,setup):
    c = 2*(f-0.5)
    c_u = 2*f_u
    c,c_u = pa.do_carbon_ROC_1q(c,c_u,setup)
    return (c/2)+0.5,c_u/2

def get_MBI_data(tstamp,ssrofolder,setup):
    if setup == 'lt3':
        base_folder = base_folder_lt3
    elif setup == 'lt4':
        base_folder = base_folder_lt4
    else:
        raise ValueError('No setup specified in get_MBI_data')

    ### note that the data is not carbon read-out corrected.
    folder = tb.data_from_time(tstamp,folder = base_folder)
    #     ssro_folder = tb.data_from_time(ssrostamp)
    a = mbi.MBIAnalysis(folder)
    a.get_sweep_pts()
    a.get_readout_results(name='adwindata')
    a.get_electron_ROC(ssrofolder)
    a.p0,a.u_p0 = do_carbon_ROC(a.p0.reshape(-1)[:],a.u_p0.reshape(-1)[:],setup) #13C ROC
    return a.sweep_pts.reshape(-1)[:],a.p0,a.u_p0

def interpolate(x_in,y,y_u,x_ref):
    """
    Interpolates data such that new sweep points match a reference (x_ref)
    """
    new_y = np.zeros(len(x_ref))
    new_y_u = np.zeros(len(x_ref))

    new_y = np.interp(x_ref,x_in,y)
    new_y_u = np.interp(x_ref,x_in,y_u)
    
    print new_y,new_y_u
    return x_ref,new_y,new_y_u

In [ ]:
### calc averages for bloch sphere
def return_avgs(Tstamps, ssro,setup):
    x_list,y_list,y_u_list = [],[],[]
    for t in Tstamps:
        x,y,y_u = get_MBI_data(t,ssro,setup)
        x_list.append(x);y_list.append(y);y_u_list.append(y_u)
        
       
    if len(x_list[0]) != len(x_list[1]): ### lt3 has a measurement with deviating number of points
        ## interpolate results:
        x_list[0],y_list[0],y_u_list[0] = interpolate(x_list[0],y_list[0],y_u_list[0],x_list[1])

    ### calculate average state fidelity
    avg_yXY = np.zeros(len(x_list[1]))
    avg_yXY_u = np.zeros(len(x_list[1]))
    avg_yZ = np.zeros(len(x_list[-1]))
    avg_yZ_u = np.zeros(len(x_list[-1]))

    ## get the avg XY values
    for y,y_u in zip(y_list[:-2],y_u_list[:-2]): ## assumption: all sweep points are the same
        avg_yXY = avg_yXY+(np.absolute(2*(y-0.5))/2+0.5)/4.
        avg_yXY_u = avg_yXY_u+y_u**2/16.

        ## get the avg Z values
    for y,y_u in zip(y_list[-2:],y_u_list[-2:]): 
        avg_yZ = avg_yZ+(np.absolute(2*(y-0.5))/2.+0.5)/2.
        avg_yZ_u = avg_yZ_u+y_u**2/4.    

    avg_yXY_u = np.sqrt(avg_yXY_u)
    avg_yZ_u = np.sqrt(avg_yZ_u)
    
    return x_list,y_list,y_u_list,avg_yXY,avg_yXY_u,avg_yZ,avg_yZ_u

In [ ]:
x_list4,y_list4,y_u_list4,avg_yXY4,avg_yXY_u4,avg_yZ4,avg_yZ_u4 = return_avgs(LT4_tstamp_list,LT4_ssro,'lt4')
x_list3,y_list3,y_u_list3,avg_yXY3,avg_yXY_u3,avg_yZ3,avg_yZ_u3 = return_avgs(LT3_tstamp_list,LT3_ssro,'lt3')

In [ ]:
### fit XY decays:
# p0,fitfunc,fitfunc_str = common.fit_exp_decay_with_offset(0.5, 0.45, 300);fixed = [0]
p0,fitfunc,fitfunc_str = common.fit_general_exponential(0.5,0.45,0,250,1.4);fixed = [0,2]
fit_result4 = fit.fit1d(x_list4[0],avg_yXY4, None, p0=p0, fitfunc=fitfunc, do_print=True, ret=True,fixed=fixed)
fit_x4 = np.linspace(x_list4[0][0],x_list4[0][-1],600)
p0,fitfunc,fitfunc_str = common.fit_general_exponential(0.5,0.45,0,250,1.2);fixed = [0,2]
fit_result3 = fit.fit1d(x_list3[0],avg_yXY3, None, p0=p0, fitfunc=fitfunc, do_print=True, ret=True,fixed=fixed)
fit_x3 = np.linspace(x_list3[0][0],x_list3[0][-1],600)

In [ ]:
save_folder = r'K:\ns\qt\Diamond\Projects\Purification\Paper\Plots' 
### plot decays
golden_ratio = 1.61803
mm_to_inches = 0.0393701 ### inch/mm
figure_width = mm_to_inches*55
fig = plt.figure(figsize =(figure_width,figure_width/golden_ratio))
ax = plt.subplot(111)
### formatting
lw = 1.
fontsize = 11.
markersize = 4.

### colors
col_XY = '#3594F2'
# green = np.array([79/255.,166/255.,58/255.]) ### the color col_XY expressed in RGB values
blue = np.array([53,148,242])/255.
green = blue
white = np.array([1,1,1])
x_max4 = x_list4[0][-1]
x_max3 = x_list3[0][-1]
col_Z = '#82DB81' #130/219/129
orange_z = np.array([130,219,129])/255.# This is orange --> np.array([242,147,53])/255.
color_gradient4 = [(1-float(x)/x_max4)*green+(float(x)/x_max4)*white for x in x_list4[0]]
color_gradient24 = [(1-float(x)/x_max4)*orange_z+(float(x)/x_max4)*orange_z for x in x_list4[0]]
color_gradient3 = [(1-float(x)/x_max3)*green+(float(x)/x_max3)*white for x in x_list3[0]]
color_gradient23 = [(1-float(x)/x_max3)*orange_z+(float(x)/x_max3)*orange_z for x in x_list3[0]]


## apply formatting
ax.set_ylabel(r'Avg. state fidelity',size = fontsize)
ax.set_xlabel('Entangling attempts',size = fontsize)
ax.yaxis.set_ticks([0.4,0.6,0.8,1.0])
ax.xaxis.set_ticks([0,100,200,300,400])
ax.tick_params(labelsize = fontsize,width =lw)
ax.set_ylim([0.5,1])
ax.set_xlim([0,450])
[i.set_linewidth(lw) for i in ax.spines.itervalues()]


### plot data
ax.scatter(x_list4[0],avg_yXY4, c=color_gradient4, s=markersize**2, zorder=3,linewidths=lw/2.)
ax.errorbar(x_list4[0],avg_yXY4,avg_yXY_u4,fmt='.', markersize = 0,zorder=0,ecolor=col_XY,mew=lw/2.)
ax.errorbar(x_list4[-1],avg_yZ4,avg_yZ_u4,fmt = 'o',ms = markersize,color=col_Z,mew=lw/2.,zorder = 1)
ax.plot(fit_x4,fit_result4['fitfunc'](fit_x4),'-',color=col_XY,lw=lw,zorder = 0)

ax.scatter(x_list3[0],avg_yXY3, c=color_gradient3,marker = '^', s=markersize**2, zorder=3,linewidths=lw/2.)
ax.errorbar(x_list3[0],avg_yXY3,avg_yXY_u3,fmt='.', zorder=0, markersize = 0,ecolor=col_XY,mew=lw/2.)
ax.errorbar(x_list3[-1],avg_yZ3,avg_yZ_u3,fmt = '^',ms = markersize,color=col_Z,mew=lw/2.,zorder = 1)
ax.plot(fit_x4,fit_result3['fitfunc'](fit_x4),'-',color=col_XY,lw=lw,zorder = 0)
plt.savefig(os.path.join(save_folder,'memory_decay_comb.png'),format='png',bbox_inches = 'tight',pad_inches=0.3)
plt.savefig(os.path.join(save_folder,'memory_decay_comb.pdf'),format='pdf',bbox_inches = 'tight',pad_inches=0.3)

In [ ]:
## time stamp ordering: X,mX,Y,mY,Z,mZ
i = 2
print x_list4[i]
print 1-np.round(y_list4[i],2)
print np.round(y_u_list4[i],2)

In [ ]:
### bloch sphere LT3
from analysis.lib.purification import blochsphere_qutip as bl; reload(bl)
### convert all y entries to bloch vector coordinates
def plot_bloch(y_list,color_gradient,color_gradient2,name,marker = 'o'):
    bv_list = [2*(y-0.5) for y in y_list]


    b = bl.Bloch(figsize=[1.2,1.2])
    # b = bl.Bloch(figsize=[3.2,3.2])
    ## X values
    b.add_points([np.zeros(len(bv_list[4])),np.zeros(len(bv_list[4])),bv_list[4]],color2=True,invert_colors = False)
    b.add_points([np.zeros(len(bv_list[5])),np.zeros(len(bv_list[5])),bv_list[5]],color2=True,invert_colors = True)
    ## Y values
    b.add_points([-bv_list[2],np.zeros(len(bv_list[2])),np.zeros(len(bv_list[2]))])
    b.add_points([-bv_list[3][::-1],np.zeros(len(bv_list[3]))[::-1],np.zeros(len(bv_list[3]))[::-1]],invert_colors = True)
    ## Z values
    b.add_points([np.zeros(len(bv_list[0])),bv_list[0],np.zeros(len(bv_list[0]))],invert_colors = False)
    b.add_points([np.zeros(len(bv_list[1])),bv_list[1][::-1],np.zeros(len(bv_list[1]))],invert_colors = True)

    ## styling
    b.frame_alpha = 0.01
    b.point_marker = marker
    b.point_color  = color_gradient
    b.point_color2 = color_gradient2
    b.point_size   = [9]
    b.zlabel = ["$\\left| 0 \\right>$",""]#["X",""]
    b.ylabel = ["$\\left| X \\right>$",""]
    b.xlabel = ["","$\\left| Y \\right>$"]

    b.save(name =os.path.join(save_folder,'bloch_'+name+'.pdf'),format = 'pdf') 
#     b.save(name =os.path.join(save_folder,'bloch_'+name+'.png'),format = 'png')
#     b.show()


In [ ]:
plot_bloch(y_list3,color_gradient3,color_gradient23,'LT3',marker = '^')

In [ ]:
plot_bloch(y_list4,color_gradient4,color_gradient24,'LT4')

# Feedback vs. No feedback

In [ ]:
#### LT3

from analysis.scripts.mbi import mbi_data_general_fits as fits; reload(fits)
####
#parameters / timestamps
tstamp_feedback = '20160817_135340'
tstamp_no_feedback = '20160817_132707'
####
Result_w_feedback = fits.exp_sin(timestamp=  tstamp_feedback,
                      offset=[0.5], amplitude = [0.5], center = [0], decay_constant = [300], 
                      exp_power = [2],
                      frequency = [0.], phase =[0], 
                      fixed = [0,2,4,5,6], base_folder = base_folder_lt3,
                      ylim = None,ssro_folder = LT3_ssro,
                      plot_fit = True, do_print = True, show_guess = False)[0]
Result_wo_feedback = fits.exp_sin(timestamp=  tstamp_no_feedback,
                      offset=[0.5], amplitude = [0.5], center = [0], decay_constant = [300], 
                      exp_power = [2],base_folder = base_folder_lt3,
                      frequency = [0.04], phase =[0], 
                      fixed = [0,2,4],
                      ylim = None,ssro_folder = LT3_ssro,
                      plot_fit = True, do_print = True, show_guess = False)[0]

In [ ]:
save_folder = r'K:\ns\qt\Diamond\Projects\Purification\Paper\Plots' 
golden_ratio = 1.61803
mm_to_inches = 0.0393701 ### inch/mm
figure_width = mm_to_inches*45
fig = plt.figure(figsize =(figure_width*1.5,figure_width/golden_ratio))
ax = plt.subplot(111)
x_wo_feed = np.linspace(Result_wo_feedback['x'][0],200,600)
x_w_feed = np.linspace(Result_w_feedback['x'][0],200,600)

### formatting
lw = 1
fontsize = 11
markersize = 4

col_w_feed = '#3594F2'
col_wo_feed = '#F29335'

ax.set_ylabel(r'p($|$0$\rangle$)',size = fontsize)
ax.set_xlabel('Entangling attempts',size = fontsize)
# ax.set_xticklabels([0,20,40,60,80,100],size = fontsize)
ax.yaxis.set_ticks([0.0,0.5,1.0])
# ax.set_yticklabels([0.0,0.5,1.0])
ax.xaxis.set_tick_params(width=lw,labelsize = fontsize)
ax.yaxis.set_tick_params(width=lw,labelsize = fontsize)
ax.set_xlim(0,100)
ax.set_ylim([0,1])
[i.set_linewidth(lw) for i in ax.spines.itervalues()]
ax.yaxis.set_label_coords(-0.23,0.5)
### commencde plotting
plt.plot(x_w_feed,Result_w_feedback['fitfunc'](x_w_feed),color = col_w_feed,lw=lw)
plt.plot(x_wo_feed,Result_wo_feedback['fitfunc'](x_wo_feed), color = col_wo_feed,lw = lw)
plt.errorbar(Result_wo_feedback['x'],Result_wo_feedback['y'],Result_wo_feedback['y_u'],fmt = 'o',color = col_wo_feed,ms = markersize,mew=lw/2.,elinewidth=lw)
plt.errorbar(Result_w_feedback['x'],Result_w_feedback['y'],Result_w_feedback['y_u'],fmt = 'o',color = col_w_feed,ms = markersize,mew=lw/2.,elinewidth=lw)
plt.savefig(os.path.join(save_folder,'feedback_nofeedback_LT3.png'),format='png',bbox_inches = 'tight',pad_inches=0.3)
plt.savefig(os.path.join(save_folder,'feedback_nofeedback_LT3.pdf'),format='pdf',bbox_inches = 'tight',pad_inches=0.3)

In [ ]:
#### LT4
from analysis.scripts.mbi import mbi_data_general_fits as fits; reload(fits)
####
#parameters / timestamps
tstamp_feedback = '20160817_141641'
tstamp_no_feedback = '20160817_132726'
####
Result_wo_feedback = fits.exp_sin(timestamp=  tstamp_no_feedback,
                      offset=[0.5], amplitude = [0.5], center = [0], decay_constant = [300], 
                      exp_power = [2],ssro_folder = LT4_ssro,
                      frequency = [0.04],phase =[0], 
                      fixed = [0,2,4],
                      ylim = None,base_folder = base_folder_lt4,
                      plot_fit = True, do_print = True, show_guess = False)[0]
Result_w_feedback = fits.exp_sin(timestamp=  tstamp_feedback,
                      offset=[0.5], amplitude = [0.5], center = [0], decay_constant = [300], 
                      exp_power = [2],ssro_folder = LT4_ssro, base_folder = base_folder_lt4,
                      frequency = [0.], phase =[0], 
                      fixed = [0,2,4,5,6],
                      ylim = None,
                      plot_fit = True, do_print = True, show_guess = False)[0]


In [ ]:
save_folder = r'K:\ns\qt\Diamond\Projects\Purification\Paper\Plots' 
golden_ratio = 1.61803
mm_to_inches = 0.0393701 ### inch/mm
figure_width = mm_to_inches*45
fig = plt.figure(figsize =(figure_width*1.5,figure_width/golden_ratio))
ax = plt.subplot(111)
x_wo_feed = np.linspace(Result_wo_feedback['x'][0],100,600)
x_w_feed = np.linspace(Result_w_feedback['x'][0],100,600)

### formatting
lw = 1
fontsize = 11
markersize = 4

col_w_feed = '#3594F2'
col_wo_feed = '#F29335'

ax.set_ylabel(r'p($|$0$\rangle$)',size = fontsize)
ax.set_xlabel('Entangling attempts',size = fontsize)
ax.set_xticklabels([0,20,40,60,80,100],size = fontsize)
ax.yaxis.set_ticks([0.0,0.5,1.0])
ax.set_yticklabels([0.0,0.5,1.0],size = fontsize)
ax.xaxis.set_tick_params(width=lw)
ax.yaxis.set_tick_params(width=lw)
ax.set_ylim([0,1])
[i.set_linewidth(lw) for i in ax.spines.itervalues()]
ax.yaxis.set_label_coords(-0.23,0.5)
### commencde plotting
plt.plot(x_w_feed,Result_w_feedback['fitfunc'](x_w_feed),color = col_w_feed,lw=lw)
plt.plot(x_wo_feed,Result_wo_feedback['fitfunc'](x_wo_feed), color = col_wo_feed,lw = lw)
plt.errorbar(Result_wo_feedback['x'],Result_wo_feedback['y'],Result_wo_feedback['y_u'],fmt = 'o',color = col_wo_feed,ms = markersize,mew=lw/2.,elinewidth=lw)
plt.errorbar(Result_w_feedback['x'],Result_w_feedback['y'],Result_w_feedback['y_u'],fmt = 'o',color = col_w_feed,ms = markersize,mew=lw/2.,elinewidth=lw)
plt.savefig(os.path.join(save_folder,'feedback_nofeedback_LT4.png'),format='png',bbox_inches = 'tight',pad_inches=0.3)
plt.savefig(os.path.join(save_folder,'feedback_nofeedback_LT4.pdf'),format='pdf',bbox_inches = 'tight',pad_inches=0.3)

In [ ]:
print Result_wo_feedback['x']
print np.round(Result_wo_feedback['y'],2)
print np.round(Result_wo_feedback['y_u'],2)

In [ ]:
print Result_w_feedback['x']
print np.round(Result_w_feedback['y'],2)
print np.round(Result_w_feedback['y_u'],2)